<a href="https://colab.research.google.com/github/TylerFlar/Cache-Simulator/blob/main/Cache-Simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import Libraries

In [2]:
import math
from datetime import datetime
import csv

# Define Your Achitecture

Remember the formula: $\text{Cache Size} = \text{Associativity} \times \text{Block Size} \times \text{Sets}$



* **Address Size:** Number of bits in an address (usually 32bit or 64bit)
* **Cache Size:** Total size of cache in bytes.
* **Block Size:** Number of bytes per line.
* **Associativity:** (a.k.a Way-Associativity): Number of parallel cache lines.  
* **Sets:** Number of lines in cache per "way."

In [7]:
"""
EDIT THE PARAMATERS OR LEAVE AS DEFAULT
"""

ADDRESS_SIZE = 64
CACHE_SIZE = 32 * 1024
BLOCK_SIZE = 64
ASSOCIATIVITY = 8
SETS = int(CACHE_SIZE / (BLOCK_SIZE * ASSOCIATIVITY))

# Setup Cache Simulator

### Cache Memory Structure:
1. **OFFSET_BITS:**
This calculates the number of bits needed to represent an offset within a block of the cache. This is done by taking the base-2 logarithm of the block size.
1. **INDEX_BITS:**
This calculates the number of bits needed to represent the index of a set within the cache. The number of sets is determined by the total cache size divided by the size of each set (BLOCK_SIZE * ASSOCIATIVITY). The base-2 logarithm of this value gives the number of bits needed.
1. **TAG_BITS:**
The remaining bits in the address after accounting for offset and index bits are used for the tag. This ensures that each memory address can be broken down into a tag, an index, and an offset.

### Cache Access Counters:
1. **total_accesses, total_hits, total_misses:**
These are counters to keep track of the total number of cache accesses, hits (successful finds in the cache), and misses (unsuccessful finds) respectively.
1. **compulsory_misses, capacity_misses, conflict_misses:** These counters segregate the types of misses:
  * **Compulsory Miss:** The first time a particular piece of data is accessed and isn't in the cache.
  * **Capacity Miss:** Occurs because the cache cannot contain all the blocks needed during execution due to its limited size.
  * **Conflict Miss:** Occurs if multiple locations in the main memory map to the same location in the cache.

### Cache Data Structure:
1. **class Entry:**
Represents a single entry in the cache, containing the tag of the memory address, the index (which set it belongs to), and a time timestamp to determine when it was last accessed. This timestamp can be used for cache replacement policies like LRU (Least Recently Used).
1. **cache:**
Represents the cache itself. It's a list of lists, where each inner list is a "set" in a set-associative cache. The number of sets is determined by the SETS value.
1. **conflict_archive, capacity_archive:**
Lists that store cache entries which were evicted due to conflict misses and capacity misses, respectively.

### Data Recording:
1. **create_csv:**
A function that accepts a filename and a list of data to create a CSV file. This CSV file will contain information about each cache access, whether it was a hit or a miss, and the type of miss if applicable.
1. **csv_data:**
A list to store information about each cache access for later writing to the CSV.

### Cache Access Function:
1. **handle_address:**
This function simulates accessing the cache with a given memory address. It breaks down the address into its tag, index, and offset using the previously calculated bit values. Then it checks if the address is in the cache (a hit) or not (a miss).
  * If it's a hit, the hit counter is incremented.
  * If it's a miss, it determines the type of miss, updates the appropriate counters, and manages adding/removing entries in/from the cache based on the cache's set associativity and potential full sets.
  
For each cache access (hit or miss), information about the access is appended to the csv_data list for future recording in a CSV file.

In [8]:
"""
DO NOT TOUCH
"""

OFFSET_BITS = int(math.log2(BLOCK_SIZE))
INDEX_BITS = int(math.log2(CACHE_SIZE / (BLOCK_SIZE * ASSOCIATIVITY)))
TAG_BITS = ADDRESS_SIZE - OFFSET_BITS - INDEX_BITS

total_accesses = 0
total_hits = 0
total_misses = 0
compulsory_misses = 0
capacity_misses = 0
conflict_misses = 0

class Entry:
    def __init__(self, tag, index, time):
        self.tag = tag
        self.index = index
        self.time = time

cache = [[] for i in range(SETS)]
conflict_archive = []
capacity_archive = []

def create_csv(filename, data):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = ['array element', 'memory address', 'tag', 'index', 'hit/miss', 'miss type']
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for row in data:
            writer.writerow(row)

csv_data = []

def handle_address(address, array_name, array_index):
    global total_accesses
    global total_hits
    global total_misses
    global compulsory_misses
    global capacity_misses
    global conflict_misses

    total_accesses += 1  # Incrementing the total accesses

    address_bin = bin(address)[2:].zfill(ADDRESS_SIZE)
    tag = int(address_bin[:-OFFSET_BITS - INDEX_BITS], 2)
    index = int(address_bin[-OFFSET_BITS - INDEX_BITS:-OFFSET_BITS], 2)
    offset = int(address_bin[-OFFSET_BITS:], 2)

    hit_or_miss = "miss"
    miss_type = "N/A"

    # Check for hit
    for table in cache:
        for entry in table:
            if entry.index == index and entry.tag == tag:
                total_hits += 1
                entry.time = datetime.now()
                hit_or_miss = "hit"
                break
        if hit_or_miss == "hit":
            break

    if hit_or_miss == "miss":
        total_misses += 1
        table = cache[index]
        # Check if the set is not full
        if len(table) < ASSOCIATIVITY:
            table.append(Entry(tag, index, datetime.now()))
            if any(entry.index == index and entry.tag == tag for entry in capacity_archive):
                capacity_misses += 1
                miss_type = "capacity"
            elif any(entry.index == index and entry.tag == tag for entry in conflict_archive):
                conflict_misses += 1
                miss_type = "conflict"
            else:
                compulsory_misses += 1
                miss_type = "compulsory"
        else:
            oldest_entry = min(table, key=lambda x: x.time)
            table.remove(oldest_entry)
            conflict_archive.append(oldest_entry)
            table.append(Entry(tag, index, datetime.now()))
            if any(entry.index == index and entry.tag == tag for entry in capacity_archive):
                capacity_misses += 1
                miss_type = "capacity"
            elif any(entry.index == index and entry.tag == tag for entry in conflict_archive):
                conflict_misses += 1
                miss_type = "conflict"
            else:
                compulsory_misses += 1
                miss_type = "compulsory"

    # Append data for CSV
    csv_data.append({
        'array element': f"{array_name}[{array_index}]",
        'memory address': hex(address),
        'tag': hex(tag),
        'index': hex(index),
        'hit/miss': hit_or_miss,
        'miss type': miss_type
    })

# Define Code to Simulate

handle_address(uint64_t, string, int): Takes in a memory address, a label (for the CSV file) and a index (for the CSV file).

In [9]:
"""
ADD CUSTOM CODE TO SIMULATE

Example C Code:
double a[16384], b[16384], c[16384], d[16384], e[16384];
/* a = 0x10000, b = 0x20000, c = 0x30000, d = 0x40000, e = 0x50000 */
for (i = 0; i < 512; i++) {
  e[i] = (a[i] * b[i] + c[i])/d[i];
  // load a[i], b[i], c[i], d[i] and then store to e[i]
}

"""
# Starting Addresses
A_START = 0x10000
B_START = 0x20000
C_START = 0x30000
D_START = 0x40000
E_START = 0x50000

for i in range(512):
      handle_address(A_START + (i * 8), "A", i) # i * 8 because there are 8 bytes in a double and A, B, C, D, and E are arrays of doubles.
      handle_address(B_START + (i * 8), "B", i)
      handle_address(C_START + (i * 8), "C", i)
      handle_address(D_START + (i * 8), "D", i)
      handle_address(E_START + (i * 8), "E", i)

# Print and Save Results

In [ ]:
"""
DO NOT TOUCH
"""

print("Total accesses: " + str(total_accesses))
print("Total hits: " + str(total_hits))
print("Total misses: " + str(total_misses))
print("Miss rate: " + str(total_misses / total_accesses))
print("Compulsory misses: " + str(compulsory_misses))
print("Capacity misses: " + str(capacity_misses))
print("Conflict misses: " + str(conflict_misses))


csv_filename = "cache_simulation.csv"
create_csv(csv_filename, csv_data)